## Learning PySpark

### RDD 

In [1]:
from pyspark import SparkContext 
from pyspark import SQLContext
import sys
import os

In [2]:
# public SparkContext(String master,
#                     String appName,
#                     String sparkHome,
#                     scala.collection.Seq<String> jars,
#                     scala.collection.Map<String,String> environment)

## build my first app 
sc = SparkContext(appName ='First Lesson', master = 'local')
sc

<SparkContext master=local appName=First Lesson>

In [3]:
sc.appName

'First Lesson'

In [7]:
## build the sqlContext 
sqlContext = SQLContext(sc)
sqlContext

In [8]:
data_heterogenous = sc.parallelize([('Ferrari', 'fast'), {'Porsche': 100000}, ['Spain','visited', 4504]]).collect()
data_heterogenous

[('Ferrari', 'fast'), {'Porsche': 100000}, ['Spain', 'visited', 4504]]

In [9]:
print(data_heterogenous[2][1], data_heterogenous[1]['Porsche'])

visited 100000


In [10]:
## read a database
# when you read a local data, you should add 'file://' at the beginning of the path
data_from_file = sc.textFile(
        'file:///Users/tjmask/Desktop/Semester 2/Spark/PySpark/Datasets/VS14MORT.txt', 
        4)

In [11]:
data_from_file.take(1)

['                   1                                          2101  M1087 432311  4M4                2014U7CN                                    I64 238 070   24 0111I64                                                                                                                                                                           01 I64                                                                                                  01  11                                 100 601']

In [12]:
def extractInformation(row):
    import re
    import numpy as np

    selected_indices = [
         2,4,5,6,7,9,10,11,12,13,14,15,16,17,18,
         19,21,22,23,24,25,27,28,29,30,32,33,34,
         36,37,38,39,40,41,42,43,44,45,46,47,48,
         49,50,51,52,53,54,55,56,58,60,61,62,63,
         64,65,66,67,68,69,70,71,72,73,74,75,76,
         77,78,79,81,82,83,84,85,87,89
    ]
    
    record_split = re\
        .compile(
            r'([\s]{19})([0-9]{1})([\s]{40})([0-9\s]{2})([0-9\s]{1})([0-9]{1})([0-9]{2})' + 
            r'([\s]{2})([FM]{1})([0-9]{1})([0-9]{3})([0-9\s]{1})([0-9]{2})([0-9]{2})' + 
            r'([0-9]{2})([0-9\s]{2})([0-9]{1})([SMWDU]{1})([0-9]{1})([\s]{16})([0-9]{4})' +
            r'([YNU]{1})([0-9\s]{1})([BCOU]{1})([YNU]{1})([\s]{34})([0-9\s]{1})([0-9\s]{1})' +
            r'([A-Z0-9\s]{4})([0-9]{3})([\s]{1})([0-9\s]{3})([0-9\s]{3})([0-9\s]{2})([\s]{1})' + 
            r'([0-9\s]{2})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([\s]{36})([A-Z0-9\s]{2})([\s]{1})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([\s]{1})([0-9\s]{2})([0-9\s]{1})' + 
            r'([0-9\s]{1})([0-9\s]{1})([0-9\s]{1})([\s]{33})([0-9\s]{3})([0-9\s]{1})([0-9\s]{1})')
    try:
        rs = np.array(record_split.split(row))[selected_indices]
    except:
        rs = np.array(['-99'] * len(selected_indices))
    return rs

In [20]:
data_from_file_conv = data_from_file.map(extractInformation)
data_from_file_conv.map(lambda row: row).take(1)

[array(['1', '  ', '2', '1', '01', 'M', '1', '087', ' ', '43', '23', '11',
        '  ', '4', 'M', '4', '2014', 'U', '7', 'C', 'N', ' ', ' ', 'I64 ',
        '238', '070', '   ', '24', '01', '11I64  ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '01',
        'I64  ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '01', ' ',
        ' ', '1', '1', '100', '6'], dtype='<U40')]

In [14]:
data_2014 = data_from_file_conv.map(lambda row: int(row[16]))
data_2014.take(10)

[2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, -99]

In [15]:
data_2014_2 = data_from_file_conv.map(lambda row: (row[16], int(row[16])))
data_2014_2.take(10)

[('2014', 2014),
 ('2014', 2014),
 ('2014', 2014),
 ('2014', 2014),
 ('2014', 2014),
 ('2014', 2014),
 ('2014', 2014),
 ('2014', 2014),
 ('2014', 2014),
 ('-99', -99)]

In [16]:
data_from_file_conv.take(1)[0][16]

'2014'

In [18]:
data_filtered = data_from_file_conv.filter(lambda row: row[5] == 'F' and row[21] == '0')
### ????????????????
# data_filtered.count()
# data_filtered.collect()
# data_filtered.take(1)

In [21]:
data_2014_flat = data_from_file_conv.flatMap(lambda row: (row[16], int(row[16]) + 1))
data_2014_flat.take(10)

['2014', 2015, '2014', 2015, '2014', 2015, '2014', 2015, '2014', 2015]

In [22]:
data_2014_flat = data_from_file_conv.flatMap(lambda row: (row[16], int(row[16])+2))
data_2014_flat.take(10)

['2014', 2016, '2014', 2016, '2014', 2016, '2014', 2016, '2014', 2016]

In [32]:
from time import *
start_time = time()
distinct_gender = data_from_file_conv.map(lambda row: row[5]).distinct().collect()
end_time = time()
run_time = end_time - start_time
print(run_time, distinct_gender)

91.19847989082336 ['-99', 'M', 'F']


In [42]:
fraction = 0.001
data_sample = data_from_file_conv.sample(False, fraction,666)
data_sample.take(1) 

[array(['1', '  ', '3', '1', '03', 'F', '1', '088', ' ', '43', '23', '11',
        '  ', '6', 'W', '2', '2014', 'U', '7', 'C', 'N', ' ', ' ', 'F03 ',
        '175', '111', '   ', '37', '02', '11F03  ', '61J189 ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '02',
        'F03  ', 'J189 ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '01', ' ',
        ' ', '1', '1', '100', '6'], dtype='<U40')]

In [43]:
data_sample.count()

2685

In [ ]:
## so giant? why so slow?!
# print('Original dataset: {0}, sample: {1}'.format(data_from_file_conv.count(), data_sample.count()))

In [44]:
rdd1 = sc.parallelize([('a', 1), ('b', 4), ('c',10)])
rdd2 = sc.parallelize([('a', 4), ('a', 1), ('b', '6'), ('d', 15)])

rdd3 = rdd1.leftOuterJoin(rdd2)
rdd3.take(5)

[('b', (4, '6')), ('c', (10, None)), ('a', (1, 4)), ('a', (1, 1))]

In [49]:
print(rdd1.collect(),'======', rdd2.collect())

[('a', 1), ('b', 4), ('c', 10)] ====== [('a', 4), ('a', 1), ('b', '6'), ('d', 15)]


In [50]:
rdd4 = rdd1.rightOuterJoin(rdd2)
rdd4.collect()

[('b', (4, '6')), ('d', (None, 15)), ('a', (1, 4)), ('a', (1, 1))]

In [51]:
rdd5 = rdd1.join(rdd2)
rdd5.collect()

[('b', (4, '6')), ('a', (1, 4)), ('a', (1, 1))]

In [52]:
rdd6 = rdd1.intersection(rdd2)
rdd6.collect()

[('a', 1)]

In [59]:
rdd1 = rdd1.repartition(4)
len(rdd1.collect())
len(rdd1.glom().collect())

4

In [65]:
rdd1.glom().collect()

[[('a', 1), ('b', 4), ('c', 10)], [], [], []]

In [66]:
data_take_sampled = data_from_file_conv.takeSample(False, 1, 667)
data_take_sampled

KeyboardInterrupt: 

In [4]:
data = sc.parallelize(
    [('Amber', 22), ('Alfred', 23), ('Skye',4), ('Albert', 12), 
     ('Amber', 9)])
data

ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:195

In [5]:
data.take(2)

[('Amber', 22), ('Alfred', 23)]

In [ ]:
for i in data.collect():
    print(i)

In [48]:
x = sqlContext.read.csv("/Users/tjmask/Desktop/Semester 2/Data Mining/HW1/hw1/Homework_1/Datasets/eye.csv")

In [26]:
x.describe().show()

+-------+-----+--------------------+-------------------+
|summary|  _c0|                 _c1|                _c2|
+-------+-----+--------------------+-------------------+
|  count|  180|                 181|                181|
|   mean| null|-0.00326944500000...|0.10504874055555553|
| stddev| null| 0.47220709838743213|  0.492723292491621|
|    min|  ORB|          -0.0049875|         -0.0049875|
|    max|PUPIL|                   X|                  Y|
+-------+-----+--------------------+-------------------+



In [50]:
z =  sc.textFile("/Users/tjmask/Desktop/Semester 2/Data Mining/HW1/hw1/Homework_1/Datasets/eye.csv")

In [53]:
z.take(10)

[',X,Y',
 'ORB,-0.78304,0.2793',
 'ORB,-0.75304,0.021',
 'ORB,-0.7581,-0.2394',
 'ORB,-0.75312,0.20449',
 'ORB,-0.72319,-0.28928',
 'ORB,-0.69825,0.23441',
 'ORB,-0.69825,0.2793',
 'ORB,-0.69825,0.34913',
 'ORB,-0.69825,0.42394']

In [22]:
y = sc.textFile('/Users/tjmask/Desktop/Courses/health care/hw3/VTINP16_upd.zip')

In [25]:
y.take(1)

["PK\x03\x04\x14\x00\x00\x00\x08\x00T]JN�N��6�2\x00�V�\x00\x0f\x00\x00\x00VTINP16_upd.TXT���r\x1cI�&x\x1f�y���w���v�B,\x04\x01�\x00�\x04qi)����t���3��������\x1e\x1e��_<3�\x08\x06��fj�~�������\x7f�������_��?��������?�������������\x7f�_ݿ��\x7f���잟/~wWw/�������\x1d����@�G�?���������\x18���\x16\x1f���Ň->m�q��[���o8�=�W?�W?�W?�W?�W?�W?�W?�W?�W?㫟�����g|�3��\x19_���~�W?㫟�����/W߯�X�9�����_�\x7f��\x7f����������������������]^o_��������\x1b�\x1e\x7f��������\x7fw/_���w���_^n�]�v��/���\x7f��\x7f��\x7ft�/_��\x0b~��t����~����?�^����+�\x13��\x7f�����^�~^w�/����U��\x7f�?�ۗ�?������\x1b\x11�;�\x11�&��u�~H��xp����/�E��v�ޛ��f�=\x1ac��\x1e��=I�{��v�%&�1��=8Z��>eZ5��/�B?�x\t��\x15�}�~����~�=�\x18��^�/�7.�'�\x19w���\x1f�)_\x1f�?>�?N�h\x17C�\\�\x1dm�s��-�����\x0c�����\x13\x19\x10ͯ��.����+�t���z�+��"]

In [14]:
from pyspark import findspark 
from pyspark import SparkContext 
# import SparkSession 
findspark.init() 
sc = SparkContext(appName="MyFirstApp") 
spark = SparkSession(sc) 
print("Hello World!") 
sc.close() #closing the spark session

ImportError: cannot import name 'findspark' from 'pyspark' (//anaconda3/lib/python3.7/site-packages/pyspark/__init__.py)

In [15]:
rdd1 = sc.parallelize([('a', 1), ('b', 4), ('c',10)])
rdd2 = sc.parallelize([('a', 4), ('a', 1), ('b', '6'), ('d', 15)])

rdd3 = rdd1.leftOuterJoin(rdd2)
rdd3.take(5)

[('b', (4, '6')), ('c', (10, None)), ('a', (1, 4)), ('a', (1, 1))]

In [16]:
rdd3.collect()

[('b', (4, '6')), ('c', (10, None)), ('a', (1, 4)), ('a', (1, 1))]